# Predict the Batch Performance
## By Team Kumaon

# PROBLEM STATEMENT
Data is being recorded from various condition monitoring sensors in a manufacturing plant. There are hundreds of such sensors which may be impacting the quality of final product which is getting manufactured, hence we can model this problem as a multivariate regression problem. Important point to note about real-life datasets is that they have various shifts since they are being recorded over the period of time and environment changes with time and further that is going to be the main focus of this challenge. There will be covariance shift, prior probability shift, domain shift etc present in train-test data, so proper method to overcome that has to be implemented.

## Overview
Input parameters are the sensors attached to a machine, which are 54 in number. In the dataset, they are named as x0, x1, x2 … x54.
Since this is a batch manufacturing, a batch is of a certain time period, say t0 is the batch start and t6 is the batch end. Sensor values being recorded at time instance t0, t1, t2 … t6 are labeled as x0_t0, x0_t1, x0_t2 … x0_t6 respectively in the dataset, similarly for the others. All values are numerical (float or binary), there are no string or multiclass type features.
All the sensors attached to the machine are physically independent although there can be statistical dependency present in their values.
Then there is output parameter y2 which has to be considered independently. They describe the batch performance.
Rows in the dataset represent batches. Row numbers are important as they are sorted time-wise. Although batches are considered independent to each other, you may find statistical dependency among them, hence you may convert this problem into a time series multivariate regression problem. It is completely your decision.
Batch performance is related to the machine’s state which is being captured by those 54 sensors. Hence this problem can be modeled as a multivariate regression problem where you train a model by looking at the machine’s state and compute the batch performance.

## Objective
Achieve the highest possible R2 score for label y2. You can shuffle the data while train-test splitting.

## Settings

Train-test split ratio has been set to 90:10 for final submission. You can choose your values for train-val.

In [24]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
#reading csv file
train=pd.read_csv('train.csv')
train.head()

,Id,x0_t0,x0_t1,x0_t2,x0_t3,x0_t4,x0_t5,x0_t6,x1_t0,x1_t1,x1_t2,x1_t3,x1_t4,x1_t5,x1_t6,x2_t0,x2_t1,x2_t2,x2_t3,x2_t4,x2_t5,x2_t6,x3_t0,x3_t1,x3_t2,x3_t3,x3_t4,x3_t5,x3_t6,x4_t0,x4_t1,x4_t2,x4_t3,x4_t4,x4_t5,x4_t6,x5_t0,x5_t1,x5_t2,x5_t3,x5_t4,x5_t5,x5_t6,x6_t0,x6_t1,x6_t2,x6_t3,x6_t4,x6_t5,x6_t6,x7_t0,x7_t1,x7_t2,x7_t3,x7_t4,x7_t5,x7_t6,x8_t0,x8_t1,x8_t2,x8_t3,x8_t4,x8_t5,x8_t6,x9_t0,x9_t1,x9_t2,x9_t3,x9_t4,x9_t5,x9_t6,x10_t0,x10_t1,x10_t2,x10_t3,x10_t4,x10_t5,x10_t6,x11_t0,x11_t1,x11_t2,x11_t3,x11_t4,x11_t5,x11_t6,x12_t0,x12_t1,x12_t2,x12_t3,x12_t4,x12_t5,x12_t6,x13_t0,x13_t1,x13_t2,x13_t3,x13_t4,x13_t5,x13_t6,x14_t0,x14_t1,x14_t2,x14_t3,x14_t4,x14_t5,x14_t6,x15_t0,x15_t1,x15_t2,x15_t3,x15_t4,x15_t5,x15_t6,x16_t0,x16_t1,x16_t2,x16_t3,x16_t4,x16_t5,x16_t6,x17_t0,x17_t1,x17_t2,x17_t3,x17_t4,x17_t5,x17_t6,x18_t0,x18_t1,x18_t2,x18_t3,x18_t4,x18_t5,x18_t6,x19_t0,x19_t1,x19_t2,x19_t3,x19_t4,x19_t5,x19_t6,x20_t0,x20_t1,x20_t2,x20_t3,x20_t4,x20_t5,x20_t6,x21_t0,x21_t1,x21_t2,x21_t3,x21_t4,x21_t5,x21_t6,x22_t0,x22_t1,x22_t2,x22_t3,x22_t4,x22_t5,x22_t6,x23_t0,x23_t1,x23_t2,x23_t3,x23_t4,x23_t5,x23_t6,x24_t0,x24_t1,x24_t2,x24_t3,x24_t4,x24_t5,x24_t6,x25_t0,x25_t1,x25_t2,x25_t3,x25_t4,x25_t5,x25_t6,x26_t0,x26_t1,x26_t2,x26_t3,x26_t4,x26_t5,x26_t6,x27_t0,x27_t1,x27_t2,x27_t3,x27_t4,x27_t5,x27_t6,x28_t0,x28_t1,x28_t2,x28_t3,x28_t4,x28_t5,x28_t6,x29_t0,x29_t1,x29_t2,x29_t3,x29_t4,x29_t5,x29_t6,x30_t0,x30_t1,x30_t2,x30_t3,x30_t4,x30_t5,x30_t6,x32_t0,x32_t1,x32_t2,x32_t3,x32_t4,x32_t5,x32_t6,x31_t0,x31_t1,x31_t2,x31_t3,x31_t4,x31_t5,x31_t6,x34_t0,x34_t1,x34_t2,x34_t3,x34_t4,x34_t5,x34_t6,x33_t0,x33_t1,x33_t2,x33_t3,x33_t4,x33_t5,x33_t6,x36_t0,x36_t1,x36_t2,x36_t3,x36_t4,x36_t5,x36_t6,x35_t0,x35_t1,x35_t2,x35_t3,x35_t4,x35_t5,x35_t6,x37_t0,x37_t1,x37_t2,x37_t3,x37_t4,x37_t5,x37_t6,x38_t0,x38_t1,x38_t2,x38_t3,x38_t4,x38_t5,x38_t6,x39_t0,x39_t1,x39_t2,x39_t3,x39_t4,x39_t5,x39_t6,x40_t0,x40_t1,x40_t2,x40_t3,x40_t4,x40_t5,x40_t6,x41_t0,x41_t1,x41_t2,x41_t3,x41_t4,x41_t5,x41_t6,x42_t0,x42_t1,x42_t2,x42_t3,x42_t4,x42_t5,x42_t6,x43_t0,x43_t1,x43_t2,x43_t3,x43_t4,x43_t5,x43_t6,x44_t0,x44_t1,x44_t2,x44_t3,x44_t4,x44_t5,x44_t6,x45_t0,x45_t1,x45_t2,x45_t3,x45_t4,x45_t5,x45_t6,x46_t0,x46_t1,x46_t2,x46_t3,x46_t4,x46_t5,x46_t6,x47_t0,x47_t1,x47_t2,x47_t3,x47_t4,x47_t5,x47_t6,x48_t0,x48_t1,x48_t2,x48_t3,x48_t4,x48_t5,x48_t6,x49_t0,x49_t1,x49_t2,x49_t3,x49_t4,x49_t5,x49_t6,x50_t0,x50_t1,x50_t2,x50_t3,x50_t4,x50_t5,x50_t6,x51_t0,x51_t1,x51_t2,x51_t3,x51_t4,x51_t5,x51_t6,x52_t0,x52_t1,x52_t2,x52_t3,x52_t4,x52_t5,x52_t6,x53_t0,x53_t1,x53_t2,x53_t3,x53_t4,x53_t5,x53_t6,x54_t0,x54_t1,x54_t2,x54_t3,x54_t4,x54_t5,x54_t6,y2
0,706,762.212216,753.162754,754.610620,762.787135,768.170347,769.815486,750.950000,0.902534,0.873314,0.807268,0.862909,0.836029,0.902826,1.37600,1.0,1.0,1.0,1.0,1.0,1.0,1.0,21.262724,21.044805,21.449664,20.734400,21.135345,20.868361,27.900000,6.7,6.7,6.7,6.7,6.7,6.7,6.9,748.773616,745.635196,745.722604,747.266661,753.433793,752.713959,746.300000,788.799988,788.799988,788.799988,788.799988,788.799988,788.799988,795.830000,810.299988,810.299988,810.299988,810.299988,810.299988,810.299988,816.480000,4.4,4.4,4.4,4.4,4.4,4.4,4.4,0.243978,0.244055,0.245363,0.248112,0.335132,0.257364,0.310000,81.980046,82.087889,82.129742,82.093134,82.041982,82.008582,81.490000,15.277053,13.794279,16.751010,21.203234,24.425490,26.686207,9.400000,473.514571,473.415871,471.324834,472.325497,468.656301,469.211082,421.730000,460.856047,458.579524,454.148703,451.174712,446.458846,439.673132,495.530000,0.897013,0.897303,0.897223,0.897176,0.897156,0.897131,0.900000,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1704.25,65.0,65.0,65.0,65.0,65.0,65.0,55.66,702.238299,701.240283,696.300131,694.692963,701.764607,701.260836,697.240000,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.000000,27.000000,27.015366,27.0,27.0,27.0,27.0,751.197977,758.306068,768.134158,790.719635,784.526051,775.290000,775.29,0.407567,0.810007,0.647788,0.651062

In [26]:
pd.set_option('max_columns',None)
#using this we can see all the columns

In [27]:
#checking head again
train.head()

,Id,x0_t0,x0_t1,x0_t2,x0_t3,x0_t4,x0_t5,x0_t6,x1_t0,x1_t1,x1_t2,x1_t3,x1_t4,x1_t5,x1_t6,x2_t0,x2_t1,x2_t2,x2_t3,x2_t4,x2_t5,x2_t6,x3_t0,x3_t1,x3_t2,x3_t3,x3_t4,x3_t5,x3_t6,x4_t0,x4_t1,x4_t2,x4_t3,x4_t4,x4_t5,x4_t6,x5_t0,x5_t1,x5_t2,x5_t3,x5_t4,x5_t5,x5_t6,x6_t0,x6_t1,x6_t2,x6_t3,x6_t4,x6_t5,x6_t6,x7_t0,x7_t1,x7_t2,x7_t3,x7_t4,x7_t5,x7_t6,x8_t0,x8_t1,x8_t2,x8_t3,x8_t4,x8_t5,x8_t6,x9_t0,x9_t1,x9_t2,x9_t3,x9_t4,x9_t5,x9_t6,x10_t0,x10_t1,x10_t2,x10_t3,x10_t4,x10_t5,x10_t6,x11_t0,x11_t1,x11_t2,x11_t3,x11_t4,x11_t5,x11_t6,x12_t0,x12_t1,x12_t2,x12_t3,x12_t4,x12_t5,x12_t6,x13_t0,x13_t1,x13_t2,x13_t3,x13_t4,x13_t5,x13_t6,x14_t0,x14_t1,x14_t2,x14_t3,x14_t4,x14_t5,x14_t6,x15_t0,x15_t1,x15_t2,x15_t3,x15_t4,x15_t5,x15_t6,x16_t0,x16_t1,x16_t2,x16_t3,x16_t4,x16_t5,x16_t6,x17_t0,x17_t1,x17_t2,x17_t3,x17_t4,x17_t5,x17_t6,x18_t0,x18_t1,x18_t2,x18_t3,x18_t4,x18_t5,x18_t6,x19_t0,x19_t1,x19_t2,x19_t3,x19_t4,x19_t5,x19_t6,x20_t0,x20_t1,x20_t2,x20_t3,x20_t4,x20_t5,x20_t6,x21_t0,x21_t1,x21_t2,x21_t3,x21_t4,x21_t5,x21_t6,x22_t0,x22_t1,x22_t2,x22_t3,x22_t4,x22_t5,x22_t6,x23_t0,x23_t1,x23_t2,x23_t3,x23_t4,x23_t5,x23_t6,x24_t0,x24_t1,x24_t2,x24_t3,x24_t4,x24_t5,x24_t6,x25_t0,x25_t1,x25_t2,x25_t3,x25_t4,x25_t5,x25_t6,x26_t0,x26_t1,x26_t2,x26_t3,x26_t4,x26_t5,x26_t6,x27_t0,x27_t1,x27_t2,x27_t3,x27_t4,x27_t5,x27_t6,x28_t0,x28_t1,x28_t2,x28_t3,x28_t4,x28_t5,x28_t6,x29_t0,x29_t1,x29_t2,x29_t3,x29_t4,x29_t5,x29_t6,x30_t0,x30_t1,x30_t2,x30_t3,x30_t4,x30_t5,x30_t6,x32_t0,x32_t1,x32_t2,x32_t3,x32_t4,x32_t5,x32_t6,x31_t0,x31_t1,x31_t2,x31_t3,x31_t4,x31_t5,x31_t6,x34_t0,x34_t1,x34_t2,x34_t3,x34_t4,x34_t5,x34_t6,x33_t0,x33_t1,x33_t2,x33_t3,x33_t4,x33_t5,x33_t6,x36_t0,x36_t1,x36_t2,x36_t3,x36_t4,x36_t5,x36_t6,x35_t0,x35_t1,x35_t2,x35_t3,x35_t4,x35_t5,x35_t6,x37_t0,x37_t1,x37_t2,x37_t3,x37_t4,x37_t5,x37_t6,x38_t0,x38_t1,x38_t2,x38_t3,x38_t4,x38_t5,x38_t6,x39_t0,x39_t1,x39_t2,x39_t3,x39_t4,x39_t5,x39_t6,x40_t0,x40_t1,x40_t2,x40_t3,x40_t4,x40_t5,x40_t6,x41_t0,x41_t1,x41_t2,x41_t3,x41_t4,x41_t5,x41_t6,x42_t0,x42_t1,x42_t2,x42_t3,x42_t4,x42_t5,x42_t6,x43_t0,x43_t1,x43_t2,x43_t3,x43_t4,x43_t5,x43_t6,x44_t0,x44_t1,x44_t2,x44_t3,x44_t4,x44_t5,x44_t6,x45_t0,x45_t1,x45_t2,x45_t3,x45_t4,x45_t5,x45_t6,x46_t0,x46_t1,x46_t2,x46_t3,x46_t4,x46_t5,x46_t6,x47_t0,x47_t1,x47_t2,x47_t3,x47_t4,x47_t5,x47_t6,x48_t0,x48_t1,x48_t2,x48_t3,x48_t4,x48_t5,x48_t6,x49_t0,x49_t1,x49_t2,x49_t3,x49_t4,x49_t5,x49_t6,x50_t0,x50_t1,x50_t2,x50_t3,x50_t4,x50_t5,x50_t6,x51_t0,x51_t1,x51_t2,x51_t3,x51_t4,x51_t5,x51_t6,x52_t0,x52_t1,x52_t2,x52_t3,x52_t4,x52_t5,x52_t6,x53_t0,x53_t1,x53_t2,x53_t3,x53_t4,x53_t5,x53_t6,x54_t0,x54_t1,x54_t2,x54_t3,x54_t4,x54_t5,x54_t6,y2
0,706,762.212216,753.162754,754.610620,762.787135,768.170347,769.815486,750.950000,0.902534,0.873314,0.807268,0.862909,0.836029,0.902826,1.37600,1.0,1.0,1.0,1.0,1.0,1.0,1.0,21.262724,21.044805,21.449664,20.734400,21.135345,20.868361,27.900000,6.7,6.7,6.7,6.7,6.7,6.7,6.9,748.773616,745.635196,745.722604,747.266661,753.433793,752.713959,746.300000,788.799988,788.799988,788.799988,788.799988,788.799988,788.799988,795.830000,810.299988,810.299988,810.299988,810.299988,810.299988,810.299988,816.480000,4.4,4.4,4.4,4.4,4.4,4.4,4.4,0.243978,0.244055,0.245363,0.248112,0.335132,0.257364,0.310000,81.980046,82.087889,82.129742,82.093134,82.041982,82.008582,81.490000,15.277053,13.794279,16.751010,21.203234,24.425490,26.686207,9.400000,473.514571,473.415871,471.324834,472.325497,468.656301,469.211082,421.730000,460.856047,458.579524,454.148703,451.174712,446.458846,439.673132,495.530000,0.897013,0.897303,0.897223,0.897176,0.897156,0.897131,0.900000,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1704.25,65.0,65.0,65.0,65.0,65.0,65.0,55.66,702.238299,701.240283,696.300131,694.692963,701.764607,701.260836,697.240000,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.000000,27.000000,27.015366,27.0,27.0,27.0,27.0,751.197977,758.306068,768.134158,790.719635,784.526051,775.290000,775.29,0.407567,0.810007,0.647788,0.651062

In [28]:
train.shape

(794, 387)

In [29]:
train.isnull().sum()

Id        0
x0_t0     0
x0_t1     0
x0_t2     0
x0_t3     0
         ..
x54_t3    0
x54_t4    0
x54_t5    0
x54_t6    0
y2        0
Length: 387, dtype: int64

In [30]:
train.describe()

,Id,x0_t0,x0_t1,x0_t2,x0_t3,x0_t4,x0_t5,x0_t6,x1_t0,x1_t1,x1_t2,x1_t3,x1_t4,x1_t5,x1_t6,x2_t0,x2_t1,x2_t2,x2_t3,x2_t4,x2_t5,x2_t6,x3_t0,x3_t1,x3_t2,x3_t3,x3_t4,x3_t5,x3_t6,x4_t0,x4_t1,x4_t2,x4_t3,x4_t4,x4_t5,x4_t6,x5_t0,x5_t1,x5_t2,x5_t3,x5_t4,x5_t5,x5_t6,x6_t0,x6_t1,x6_t2,x6_t3,x6_t4,x6_t5,x6_t6,x7_t0,x7_t1,x7_t2,x7_t3,x7_t4,x7_t5,x7_t6,x8_t0,x8_t1,x8_t2,x8_t3,x8_t4,x8_t5,x8_t6,x9_t0,x9_t1,x9_t2,x9_t3,x9_t4,x9_t5,x9_t6,x10_t0,x10_t1,x10_t2,x10_t3,x10_t4,x10_t5,x10_t6,x11_t0,x11_t1,x11_t2,x11_t3,x11_t4,x11_t5,x11_t6,x12_t0,x12_t1,x12_t2,x12_t3,x12_t4,x12_t5,x12_t6,x13_t0,x13_t1,x13_t2,x13_t3,x13_t4,x13_t5,x13_t6,x14_t0,x14_t1,x14_t2,x14_t3,x14_t4,x14_t5,x14_t6,x15_t0,x15_t1,x15_t2,x15_t3,x15_t4,x15_t5,x15_t6,x16_t0,x16_t1,x16_t2,x16_t3,x16_t4,x16_t5,x16_t6,x17_t0,x17_t1,x17_t2,x17_t3,x17_t4,x17_t5,x17_t6,x18_t0,x18_t1,x18_t2,x18_t3,x18_t4,x18_t5,x18_t6,x19_t0,x19_t1,x19_t2,x19_t3,x19_t4,x19_t5,x19_t6,x20_t0,x20_t1,x20_t2,x20_t3,x20_t4,x20_t5,x20_t6,x21_t0,x21_t1,x21_t2,x21_t3,x21_t4,x21_t5,x21_t6,x22_t0,x22_t1,x22_t2,x22_t3,x22_t4,x22_t5,x22_t6,x23_t0,x23_t1,x23_t2,x23_t3,x23_t4,x23_t5,x23_t6,x24_t0,x24_t1,x24_t2,x24_t3,x24_t4,x24_t5,x24_t6,x25_t0,x25_t1,x25_t2,x25_t3,x25_t4,x25_t5,x25_t6,x26_t0,x26_t1,x26_t2,x26_t3,x26_t4,x26_t5,x26_t6,x27_t0,x27_t1,x27_t2,x27_t3,x27_t4,x27_t5,x27_t6,x28_t0,x28_t1,x28_t2,x28_t3,x28_t4,x28_t5,x28_t6,x29_t0,x29_t1,x29_t2,x29_t3,x29_t4,x29_t5,x29_t6,x30_t0,x30_t1,x30_t2,x30_t3,x30_t4,x30_t5,x30_t6,x32_t0,x32_t1,x32_t2,x32_t3,x32_t4,x32_t5,x32_t6,x31_t0,x31_t1,x31_t2,x31_t3,x31_t4,x31_t5,x31_t6,x34_t0,x34_t1,x34_t2,x34_t3,x34_t4,x34_t5,x34_t6,x33_t0,x33_t1,x33_t2,x33_t3,x33_t4,x33_t5,x33_t6,x36_t0,x36_t1,x36_t2,x36_t3,x36_t4,x36_t5,x36_t6,x35_t0,x35_t1,x35_t2,x35_t3,x35_t4,x35_t5,x35_t6,x37_t0,x37_t1,x37_t2,x37_t3,x37_t4,x37_t5,x37_t6,x38_t0,x38_t1,x38_t2,x38_t3,x38_t4,x38_t5,x38_t6,x39_t0,x39_t1,x39_t2,x39_t3,x39_t4,x39_t5,x39_t6,x40_t0,x40_t1,x40_t2,x40_t3,x40_t4,x40_t5,x40_t6,x41_t0,x41_t1,x41_t2,x41_t3,x41_t4,x41_t5,x41_t6,x42_t0,x42_t1,x42_t2,x42_t3,x42_t4,x42_t5,x42_t6,x43_t0,x43_t1,x43_t2,x43_t3,x43_t4,x43_t5,x43_t6,x44_t0,x44_t1,x44_t2,x44_t3,x44_t4,x44_t5,x44_t6,x45_t0,x45_t1,x45_t2,x45_t3,x45_t4,x45_t5,x45_t6,x46_t0,x46_t1,x46_t2,x46_t3,x46_t4,x46_t5,x46_t6,x47_t0,x47_t1,x47_t2,x47_t3,x47_t4,x47_t5,x47_t6,x48_t0,x48_t1,x48_t2,x48_t3,x48_t4,x48_t5,x48_t6,x49_t0,x49_t1,x49_t2,x49_t3,x49_t4,x49_t5,x49_t6,x50_t0,x50_t1,x50_t2,x50_t3,x50_t4,x50_t5,x50_t6,x51_t0,x51_t1,x51_t2,x51_t3,x51_t4,x51_t5,x51_t6,x52_t0,x52_t1,x52_t2,x52_t3,x52_t4,x52_t5,x52_t6,x53_t0,x53_t1,x53_t2,x53_t3,x53_t4,x53_t5,x53_t6,x54_t0,x54_t1,x54_t2,x54_t3,x54_t4,x54_t5,x54_t6,y2
count,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,7.940000e+02,7.940000e+02,7.940000e+02,7.940000e+02,7.940000e+02,7.940000e+02,7.940000e+02,794.0,794.0,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,7.940000e+02,7.940000e+02,794.000000,7.940000e+02,7.940000e+02,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,7.940000e+02,794.000000,7.940000e+02,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.000000,794.

In [37]:
#let's try to get average of data collected in a week and generalize column
str1='x'
str2='_t'
train_avg=train[['Id','y2']]
for i in range(0,55):
    str_1=str1+str(i)
    add=0
    for j in range(0,7):
        str_2=str_1+str2+str(j)
        add+=train[str_2]
    train_avg[str_1]=add/7
train_avg.head()

C:\Users\Mayank\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Id,y2,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54
0,706,0.72,760.244080,0.937269,1.0,22.056471,6.728571,748.549404,789.804275,811.182847,4.4,0.269143,81.975911,18.219610,464.311165,458.060138,0.897572,1357.750000,63.665714,699.248160,80.0,80.0,1.0,27.002195,771.923413,0.620713,0.571429,0.0,11.432371,822.512086,0.142857,3875.245378,452.398282,14.892789,14.497143,35.365725,34.963383,97.788514,97.905714,0.000000,0.0,51.572556,86.043619,0.198817,1.0,0.857143,0.142857,0.000000,85.856746,0.857143,700.196065,119.060403,14.758329,878.637498,646.594635,89.693307,1.892407
1,239,0.62,757.366380,1.379567,1.0,26.895488,6.500000,749.596987,845.599976,840.799988,4.5,0.279756,76.698810,54.505696,481.326723,427.410667,0.900295,1700.000000,57.000000,696.463580,80.0,80.0,1.0,29.000000,774.974019,0.492991,0.642857,0.0,12.956240,820.825508,0.071429,4085.155006,382.562997,16.859722,14.070000,39.216508,38.602857,97.847874,97.952857,0.000000,0.0,60.232345,85.091646,0.192942,1.0,0.928571,0.071429,0.000000,84.719534,0.928571,547.953130,132.105129,19.044833,886.672018,712.281086,74.327068,1.913206
2,613,0.57,757.052327,0.864291,1.0,22.271551,6.900000,820.260866,793.147153,815.468561,4.4,0.209458,82.139933,9.297903,413.788298,433.359535,0.901479,1357.750000,64.522857,697.047573,80.0,80.0,1.0,28.020719,774.704272,0.596236,0.571429,0.0,11.748918,802.226723,0.142857,3753.099335,521.043722,17.320135,14.497143,35.175877,35.205714,97.723647,97.863713,0.142857,0.0,57.449495,90.279048,0.200046,1.0,0.142857,0.857143,0.142857,91.578217,0.571429,547.210846,109.675539,13.939582,861.906494,676.498573,87.959533,2.052945
3,588,1.01,760.571769,0.808450,1.0,28.319341,6.900000,753.283047,793.147153,815.468561,4.4,0.181063,81.888624,118.440647,421.961638,547.761377,0.901376,1700.607143,55.094286,700.842542,80.0,80.0,1.0,27.000000,717.249578,0.537579,0.428571,0.0,13.462370,793.734623,0.142857,3711.580815,512.376801,14.688665,14.497143,35.313175,35.348571,97.611068,97.695710,0.142857,0.0,52.399161,88.480221,0.196191,1.0,0.142857,0.857143,0.000000,89.831031,0.857143,630.162187,137.102413,14.898321,871.835244,653.640959,88.314801,1.940227
4,780,-0.42,768.801759,0.822851,1.0,19.236735,6.814286,760.840050,775.918551,766.868551,4.4,0.290602,82.104374,2.552850,525.866158,536.969565,0.851040,1357.750000,64.522857,697.671191,80.0,80.0,1.0,26.694875,797.721864,0.534037,0.714286,0.0,12.589026,833.061031,0.142857,3751.611019,456.618927,16.125748,14.497143,34.151372,34.634286,97.850337,97.905714,0.000000,0.0,92.823810,87.816697,0.197166,1.0,0.857143,0.142857,0.142857,88.506798,0.714286,497.926779,91.554997,14.059461,855.777351,640.448159,87.042743,2.248662


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

X=train_avg.drop(columns=['Id','y2'])
y=train_avg['y2']

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,train_size=0.9)

In [39]:
rf=RandomForestRegressor()
lr=LinearRegression()
dt=DecisionTreeRegressor()
et=ExtraTreeRegressor()
gr=GradientBoostingRegressor()

In [40]:
model_dict={
    rf:'RandomForestRegressor',
    lr:'LinearRegression',
    dt:'DecisionTreeRegressor',
    et:'ExtraTreeRegressor',
    gr:'GradientBoostingRegressor'
}

In [41]:
def model_train(model_dict,X_train,X_test,y_train,y_test):
    result=[]
    for model in model_dict:
        result_dict={}
        model.fit(X_train,y_train)
        y_pred=model.predict(X_test)
        result_dict['Model']=model_dict[model]
        result_dict['R^2']=r2_score(y_test,y_pred)
        result.append(result_dict)
    return pd.DataFrame(result)

In [42]:
model_train(model_dict,X_train,X_test,y_train,y_test)

,Model,R^2
0,RandomForestRegressor,0.388653
1,LinearRegression,0.251613
2,DecisionTreeRegressor,0.198895
3,ExtraTreeRegressor,0.015237
4,GradientBoostingRegressor,0.338560


In [43]:
test=pd.read_csv('test.csv')
test.head()

,Id,x0_t0,x0_t1,x0_t2,x0_t3,x0_t4,x0_t5,x0_t6,x1_t0,x1_t1,x1_t2,x1_t3,x1_t4,x1_t5,x1_t6,x2_t0,x2_t1,x2_t2,x2_t3,x2_t4,x2_t5,x2_t6,x3_t0,x3_t1,x3_t2,x3_t3,x3_t4,x3_t5,x3_t6,x4_t0,x4_t1,x4_t2,x4_t3,x4_t4,x4_t5,x4_t6,x5_t0,x5_t1,x5_t2,x5_t3,x5_t4,x5_t5,x5_t6,x6_t0,x6_t1,x6_t2,x6_t3,x6_t4,x6_t5,x6_t6,x7_t0,x7_t1,x7_t2,x7_t3,x7_t4,x7_t5,x7_t6,x8_t0,x8_t1,x8_t2,x8_t3,x8_t4,x8_t5,x8_t6,x9_t0,x9_t1,x9_t2,x9_t3,x9_t4,x9_t5,x9_t6,x10_t0,x10_t1,x10_t2,x10_t3,x10_t4,x10_t5,x10_t6,x11_t0,x11_t1,x11_t2,x11_t3,x11_t4,x11_t5,x11_t6,x12_t0,x12_t1,x12_t2,x12_t3,x12_t4,x12_t5,x12_t6,x13_t0,x13_t1,x13_t2,x13_t3,x13_t4,x13_t5,x13_t6,x14_t0,x14_t1,x14_t2,x14_t3,x14_t4,x14_t5,x14_t6,x15_t0,x15_t1,x15_t2,x15_t3,x15_t4,x15_t5,x15_t6,x16_t0,x16_t1,x16_t2,x16_t3,x16_t4,x16_t5,x16_t6,x17_t0,x17_t1,x17_t2,x17_t3,x17_t4,x17_t5,x17_t6,x18_t0,x18_t1,x18_t2,x18_t3,x18_t4,x18_t5,x18_t6,x19_t0,x19_t1,x19_t2,x19_t3,x19_t4,x19_t5,x19_t6,x20_t0,x20_t1,x20_t2,x20_t3,x20_t4,x20_t5,x20_t6,x21_t0,x21_t1,x21_t2,x21_t3,x21_t4,x21_t5,x21_t6,x22_t0,x22_t1,x22_t2,x22_t3,x22_t4,x22_t5,x22_t6,x23_t0,x23_t1,x23_t2,x23_t3,x23_t4,x23_t5,x23_t6,x24_t0,x24_t1,x24_t2,x24_t3,x24_t4,x24_t5,x24_t6,x25_t0,x25_t1,x25_t2,x25_t3,x25_t4,x25_t5,x25_t6,x26_t0,x26_t1,x26_t2,x26_t3,x26_t4,x26_t5,x26_t6,x27_t0,x27_t1,x27_t2,x27_t3,x27_t4,x27_t5,x27_t6,x28_t0,x28_t1,x28_t2,x28_t3,x28_t4,x28_t5,x28_t6,x29_t0,x29_t1,x29_t2,x29_t3,x29_t4,x29_t5,x29_t6,x30_t0,x30_t1,x30_t2,x30_t3,x30_t4,x30_t5,x30_t6,x32_t0,x32_t1,x32_t2,x32_t3,x32_t4,x32_t5,x32_t6,x31_t0,x31_t1,x31_t2,x31_t3,x31_t4,x31_t5,x31_t6,x34_t0,x34_t1,x34_t2,x34_t3,x34_t4,x34_t5,x34_t6,x33_t0,x33_t1,x33_t2,x33_t3,x33_t4,x33_t5,x33_t6,x36_t0,x36_t1,x36_t2,x36_t3,x36_t4,x36_t5,x36_t6,x35_t0,x35_t1,x35_t2,x35_t3,x35_t4,x35_t5,x35_t6,x37_t0,x37_t1,x37_t2,x37_t3,x37_t4,x37_t5,x37_t6,x38_t0,x38_t1,x38_t2,x38_t3,x38_t4,x38_t5,x38_t6,x39_t0,x39_t1,x39_t2,x39_t3,x39_t4,x39_t5,x39_t6,x40_t0,x40_t1,x40_t2,x40_t3,x40_t4,x40_t5,x40_t6,x41_t0,x41_t1,x41_t2,x41_t3,x41_t4,x41_t5,x41_t6,x42_t0,x42_t1,x42_t2,x42_t3,x42_t4,x42_t5,x42_t6,x43_t0,x43_t1,x43_t2,x43_t3,x43_t4,x43_t5,x43_t6,x44_t0,x44_t1,x44_t2,x44_t3,x44_t4,x44_t5,x44_t6,x45_t0,x45_t1,x45_t2,x45_t3,x45_t4,x45_t5,x45_t6,x46_t0,x46_t1,x46_t2,x46_t3,x46_t4,x46_t5,x46_t6,x47_t0,x47_t1,x47_t2,x47_t3,x47_t4,x47_t5,x47_t6,x48_t0,x48_t1,x48_t2,x48_t3,x48_t4,x48_t5,x48_t6,x49_t0,x49_t1,x49_t2,x49_t3,x49_t4,x49_t5,x49_t6,x50_t0,x50_t1,x50_t2,x50_t3,x50_t4,x50_t5,x50_t6,x51_t0,x51_t1,x51_t2,x51_t3,x51_t4,x51_t5,x51_t6,x52_t0,x52_t1,x52_t2,x52_t3,x52_t4,x52_t5,x52_t6,x53_t0,x53_t1,x53_t2,x53_t3,x53_t4,x53_t5,x53_t6,x54_t0,x54_t1,x54_t2,x54_t3,x54_t4,x54_t5,x54_t6
0,584,738.152456,747.990517,754.681726,750.607263,754.378901,746.444051,750.95,0.740932,0.706801,0.723154,0.723744,0.716378,0.740058,1.376,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28.531013,28.256823,28.316151,28.232895,28.210854,25.527715,27.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,750.314288,740.029072,743.596073,742.075970,746.370848,746.184188,746.3,792.700012,792.700012,792.700012,792.700012,792.700012,792.700012,795.83,815.299988,815.299988,815.299988,815.299988,815.299988,815.299988,816.48,4.4,4.4,4.4,4.4,4.4,4.4,4.4,0.162187,0.190441,0.204893,0.253198,0.322366,0.322622,0.31,81.090687,81.377809,81.132017,81.138713,81.241582,81.294021,81.49,124.377941,118.625122,115.636000,112.360149,109.459852,107.124760,9.4,429.430123,430.478779,428.489632,427.731708,426.914454,426.039340,421.73,551.390643,547.942011,544.605770,542.779752,535.433689,533.340814,495.53,0.901902,0.901779,0.901742,0.901700,0.901860,0.901598,0.9,1700.0,1700.0,1700.0,1700.0,1700.0,1700.0,1704.25,55.0,55.0,55.0,55.0,55.0,55.0,55.66,689.958527,694.394941,698.277671,699.610619,700.011078,699.961314,697.24,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.360976,27.101220,27.390686,27.480049,27.589163,27.190657,27.0,742.117226,754.252086,773.309806,785.818768,788.435255,775.29,775.29,0.625823,0.620459,0.643212,0.414477,0.414787,0.59,0.59,0.0,1.0,1.0,1.0,1.0,

In [45]:
str1='x'
str2='_t'
test_avg=test[['Id']]
for i in range(0,55):
    str_1=str1+str(i)
    add=0
    for j in range(0,7):
        str_2=str_1+str2+str(j)
        add+=test[str_2]
    test_avg[str_1]=add/7
test_avg.head()

C:\Users\Mayank\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Id,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54
0,584,749.029273,0.818153,1.0,27.853636,6.900000,744.981491,793.147153,815.468561,4.400000,0.252244,81.252118,99.569118,427.259148,535.860383,0.901512,1700.607143,55.094286,697.064879,80.0,80.0,1.0,27.301821,770.644735,0.556965,0.714286,0.0,11.483666,814.817778,0.142857,3823.583507,496.864468,15.583112,14.497143,35.717298,34.777143,97.558046,97.695710,0.142857,0.000000,56.948642,89.866601,0.199617,1.000000,0.142857,0.857143,0.000000,89.620102,0.714286,635.229789,131.516752,15.504052,873.518651,675.151972,89.088193,1.802130
1,678,750.867913,0.807652,1.0,22.574179,6.900000,725.922392,779.690000,769.354286,4.400000,0.286712,81.139397,15.989649,505.649731,531.501220,0.882229,1357.750000,64.522857,699.690641,80.0,80.0,1.0,29.571429,779.596059,0.602103,0.571429,0.0,11.383379,845.352453,0.142857,3862.545787,437.037392,20.151888,14.568571,35.108247,35.776435,97.770798,97.905714,0.000000,0.000000,56.349126,91.001530,0.203007,1.000000,0.142857,0.857143,0.142857,91.478412,0.857143,482.351551,127.100185,16.215841,878.471282,654.244125,88.902519,1.887200
2,728,769.521485,0.938685,1.0,20.561553,6.728571,765.828892,789.804275,811.182847,4.400000,0.241413,82.290412,46.850754,508.809270,473.692904,0.897654,1357.750000,63.665714,699.386363,80.0,80.0,1.0,27.175720,763.553285,0.580660,0.714286,0.0,10.944467,840.863981,0.142857,3238.937461,448.211463,17.899226,14.497143,34.633275,35.240675,97.821814,97.905714,0.285714,0.000000,100.487060,85.956706,0.198453,1.000000,0.857143,0.142857,0.142857,85.303790,0.857143,608.529428,105.271948,14.764415,863.001064,635.541604,88.922405,2.096705
3,338,759.605706,1.087889,1.0,27.140619,6.642857,743.527544,800.604307,821.382868,4.485714,0.160642,82.159567,31.671886,524.446410,511.683073,0.897277,1700.607143,55.951429,696.859041,80.0,80.0,1.0,27.703395,757.115718,0.903090,0.428571,0.0,12.979212,792.010747,0.142857,3581.971601,417.964923,18.221882,14.140000,38.080039,37.104274,90.112136,90.205713,0.142857,0.142857,55.429838,83.744597,0.211664,0.857143,0.142857,0.857143,0.142857,81.903817,0.857143,331.579908,102.718705,14.086082,891.416185,685.211238,85.620231,1.925087
4,570,766.018495,1.351604,1.0,21.407235,6.900000,758.086469,783.632836,772.354286,4.400000,0.146330,81.797886,24.339917,534.882910,567.042758,0.855210,1332.035714,64.522857,696.790998,80.0,80.0,1.0,27.000000,777.576342,0.599877,0.571429,0.0,10.694104,813.360817,0.142857,3733.430173,516.069855,14.615062,14.497143,35.254347,34.919596,97.558933,97.695710,0.000000,0.000000,58.123566,89.442032,0.201018,1.000000,0.142857,0.857143,0.000000,90.171724,0.857143,383.293113,119.050090,15.602263,872.386742,652.076070,89.082797,2.123448


In [46]:
x=test_avg.drop(columns='Id')

In [47]:
for model in model_dict:
    result_dict={}
    y_pred=model.predict(x)
    output=pd.concat([test['Id'],pd.DataFrame(y_pred,columns=['y2'])],axis=1)
    output.to_csv(model_dict[model]+'.csv',index=False)